In [1]:
%matplotlib inline
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None,784])
X_img = tf.reshape(X,[-1,28,28,1])
Y = tf.placeholder(tf.float32, [None,10])

W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))

In [5]:
L1 = tf.nn.conv2d(X_img , W1, strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [7]:
print(L1)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))

L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
L2 = tf.reshape(L2, [-1,7*7*64])

In [10]:
W3 = tf.get_variable("W3",shape=[7*7*64,10],initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2,W3) +b

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 15
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch =int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs , batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c,_ =sess.run([cost,optimizer],feed_dict=feed_dict)
        avg_cost += c/total_batch
    print('Epoch:', '%04d'%(epoch+1),'cost: ','{:.9f}'.format(avg_cost))
    
print('Learning Finished')

Epoch: 0001 cost:  0.147990975
Epoch: 0002 cost:  0.050808442
Epoch: 0003 cost:  0.039536985
Epoch: 0004 cost:  0.030666846
Epoch: 0005 cost:  0.028614250
Epoch: 0006 cost:  0.025062210
Epoch: 0007 cost:  0.024596642
Epoch: 0008 cost:  0.023826349
Epoch: 0009 cost:  0.021079453
Epoch: 0010 cost:  0.020169555
Epoch: 0011 cost:  0.019023634
Epoch: 0012 cost:  0.019385777
Epoch: 0013 cost:  0.024767203
Epoch: 0014 cost:  0.022669282
Epoch: 0015 cost:  0.017779978
Learning Finished


In [16]:
correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('Acc :',sess.run(accuracy,feed_dict= { X: mnist.test.images ,Y: mnist.test.labels}))

Acc : 0.9851
